---
---
---
# **Studienarbeit in der Vorlesung 'Applied Big Data Analyitics'**
# **Implementation: Percent Price Oscillator (PPO) anhand des Data-Warehouse-Models**
---
---
---
* ### **Erstellt:** Wintersemeseter 2019-2020
* ### **Dozent:** Prof. Dr. Sebastian Leuoth
* ### **Autor:** Jan Gaida
* ### **Email:** jan.gaida@hof-university.de
* ### **Git:** [trading_2019](https://github.com/sleuoth-hof/trading_2019), [JanGaida](https://github.com/JanGaida) 
---
---
### *Powered by:*
### ***Hochschule für Angewandte Wissenschaften Hof***
[![Logo: Hochschule Hof](https://www.uni-assist.de/fileadmin/_processed_/4/7/csm_hof-university_logo_308ee8b37b.jpg)](https://www.hof-university.de/)
---
---
---
*© 2019-2020 Jan Gaida, Prof. Dr. Sebastian Leuoth. All Rights Reserved.*







---
# Datenbank
---
*Urspung: [More than 400 Cryptocurrency-Chartdata](https://www.kaggle.com/tencars/392-crypto-currency-pairs-at-minute-resolution/version/2)*

In [1]:
# clonen
!git clone https://github.com/sleuoth/ABDA2019.git
!ls ABDA2019/testdaten/cryptominuteresolution

# entpacken 'btcusd'
!unzip ABDA2019/testdaten/cryptominuteresolution/btcusd.csv.zip
!mv btcusd.csv ABDA2019/testdaten/cryptominuteresolution/btcusd.csv

Cloning into 'ABDA2019'...
remote: Enumerating objects: 424, done.
remote: Total 424 (delta 0), reused 0 (delta 0), pack-reused 424
Receiving objects: 100% (424/424), 486.44 MiB | 19.19 MiB/s, done.
Resolving deltas: 100% (7/7), done.
Checking out files: 100% (418/418), done.
abseth.csv	cndeth.csv  fsnbtc.csv	mkreth.csv  repbtc.csv	utketh.csv
absusd.csv	cndusd.csv  fsneth.csv	mkrusd.csv  repeth.csv	utkusd.csv
agibtc.csv	cnneth.csv  fsnusd.csv	mlneth.csv  repusd.csv	utneth.csv
agieth.csv	cnnusd.csv  fttusd.csv	mlnusd.csv  reqbtc.csv	utnusd.csv
agiusd.csv	csxeth.csv  fttust.csv	mnabtc.csv  reqeth.csv	veebtc.csv
aidbtc.csv	csxusd.csv  funbtc.csv	mnaeth.csv  requsd.csv	veeeth.csv
aideth.csv	ctxbtc.csv  funeth.csv	mnausd.csv  rifbtc.csv	veeusd.csv
aidusd.csv	ctxeth.csv  funusd.csv	mtnbtc.csv  rifusd.csv	vetbtc.csv
aiobtc.csv	ctxusd.csv  geneth.csv	mtneth.csv  rlcbtc.csv	veteth.csv
aioeth.csv	dadbtc.csv  genusd.csv	mtnusd.csv  rlceth.csv	vetusd.csv
aiousd.csv	dadeth.csv  gnoeth.csv	ncabtc.cs

---
# Weitere Frameworkinstallation
---

In [2]:
# jdk
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# tree
!apt-get install tree

# spark-package
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

# findspark
!pip install findspark

# numpy
!pip install numpy

# timeseries library
!pip install ts ts-flint

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 7 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 1s (75.2 kB/s)
Selecting previously unselected package tree.
(Reading database ... 135124 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
  Created wheel for ts: filename=ts-0.5.1-cp36-none-any.whl size=14367 sha256=d92562f655b29e68315a801c9f5e07e59e61d9f177c9e9d5f84171b8abf1744a
  

---
# Imports
---

In [4]:
# Für Pfade und andere OS-Funktionalität
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

if 'COLAB_TPU_ADDR' not in os.environ:
  print(">> Colab verbunden mit einer TPU")
else:
  print(">> Colab nicht verbunden mit einer TPU")

# Spark und Spark-SQL
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import input_file_name, col, collect_list, concat_ws, udf
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql.window import Window

# Initialisierung Spark und Spark-SQL
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

# Für einfache Weiterverarbeitung des Datenframes (z.B. Visualisierung mit Plotty)
import pandas as pd

# Visualisierung als Diagramm
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Für Darstellung von Datenframes
from IPython.display import display, HTML

>> Colab verbunden mit einer TPU


---
# Anwendung PPO-Indikator
---

*PPO Implementation basierend auf: [Investopedia.com](https://www.investopedia.com/terms/p/ppo.asp#formula-and-calculation-for-ppo)*


In [0]:
Console_Preview_Limit = 5  #@param {type: "slider", min: 1, max: 100}
#@markdown 'Git' enstpricht den dem zuvor geladen Repository 'ABDA2019':
Datenframe_Von = "Git" #@param ["Git", "Sample: Intel (requieres './intc.csv')"] {allow-input: false}
#@markdown Nur benutzt wenn 'Datenframe_Von' auf 'Git' gesetzt ist: 
Filenamen_Zu_Laden = 'btcus*.csv'  #@param {type: "string"}



*Metadaten-Aggregation*
---
---

In [6]:
#
# Intialen Datenframe bilden
#

if Datenframe_Von == "Git":
  df_spark = spark.read.csv('./ABDA2019/testdaten/cryptominuteresolution/'+Filenamen_Zu_Laden, inferSchema = True, header = True)
  start = 59
  end = 69

elif Datenframe_Von == "Sample: Intel (requieres './intc.csv')":
  df_spark = spark.read.csv('./intc.csv', inferSchema=True, header=True)
  start = 17
  end = 26

df_spark = df_spark \
  .withColumn('filepath', input_file_name()) \
  .withColumn('filename', input_file_name()[start:end]) \
  .withColumn('timestamp', df_spark['time']/1000) \
  .withColumn('date', (df_spark['time']/1000).cast('timestamp'))

print("Type:", type(df_spark), "\n")
print("Schema: ", end = '')
df_spark.printSchema()
df_spark_total_count = df_spark.count()
print("Available rows:", df_spark_total_count, "\n")

df_spark.show(Console_Preview_Limit)

df_spark.createOrReplaceTempView("base_data")

df_spark_informationView = spark.sql("SELECT filename, count(*), min(date), max(date)  FROM base_data group by filename")
df_spark_informationView.show()

Type: <class 'pyspark.sql.dataframe.DataFrame'> 

Schema: root
 |-- time: long (nullable = true)
 |-- open: double (nullable = true)
 |-- close: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- filepath: string (nullable = false)
 |-- filename: string (nullable = false)
 |-- timestamp: double (nullable = true)
 |-- date: timestamp (nullable = true)

Available rows: 2521717 

+-------------+-----+-----+-----+-----+-----------+--------------------+----------+------------+-------------------+
|         time| open|close| high|  low|     volume|            filepath|  filename|   timestamp|               date|
+-------------+-----+-----+-----+-----+-----------+--------------------+----------+------------+-------------------+
|1364774820000|93.25| 93.3| 93.3|93.25|       93.3|file:///content/A...|btcusd.csv|1.36477482E9|2013-04-01 00:07:00|
|1364774880000|100.0|100.0|100.0|100.0|       93.3|file:///conte

*Tag-Filter*
---
---

In [7]:
#
# PRO-TAG-FILTER
#

base_data_dayfiltered = spark.sql(
  """SELECT d.* FROM base_data d  
     where  (d.date, filename) in (select max(m.date), filename from base_data m group by date_format(m.date, "y-M-d"), filename)
  """)
base_data_dayfiltered.createOrReplaceTempView("base_data_dayfiltered")
#base_data_dayfiltered.show(Console_Preview_Limit)

base_data_dayfiltered_total_count = base_data_dayfiltered.count()
print("Available rows:", base_data_dayfiltered_total_count, "(lost",(df_spark_total_count-base_data_dayfiltered_total_count), "rows)", "\n")

base_data_dayfiltered_informationView = spark.sql("SELECT filename, count(*), min(date), max(date)  FROM base_data_dayfiltered group by filename")
base_data_dayfiltered_informationView.show()

Available rows: 2403 (lost 2519314 rows) 

+----------+--------+-------------------+-------------------+
|  filename|count(1)|          min(date)|          max(date)|
+----------+--------+-------------------+-------------------+
|btcust.csv|     192|2019-03-11 23:54:00|2019-10-01 23:35:00|
|btcusd.csv|    2211|2013-04-01 23:58:00|2019-10-01 18:46:00|
+----------+--------+-------------------+-------------------+



*EMA-UDF*
---
---
Berrechnet nach: [tradistats.com](https://tradistats.com/exponentieller-gleitender-durchschnitt/)

In [0]:
#
# EMA-UDF
#

# see https://tradistats.com/exponentieller-gleitender-durchschnitt/
def ema(ar):
    if len(ar) > 0:
       SF  = 2/ (len(ar)+1)
       SFi = 1 - SF
       my_ema = ar[0]
       for i in ar:
           my_ema = (i * SF) + (my_ema * SFi)
    return my_ema

ema_udf = udf(ema, DoubleType())

*Windows*
---
---

In [0]:
#
# EMA-WINDOWS
#
win26 = (Window
    .partitionBy("filename")
    .orderBy("date") \
    .rowsBetween(-25, 0))

win12 = (Window
    .partitionBy("filename")
    .orderBy("date") \
    .rowsBetween(-11, 0))

win9 = (Window
    .partitionBy("filename")
    .orderBy("date") \
    .rowsBetween(-8, 0))

*PPO- & Signal-Berechnung*
---
---

In [0]:
#
# EMA_26 && EMA_12 CALCULATION
#

base_data_tmp_ema_1 = base_data_dayfiltered \
  .withColumn('win26_close_list', collect_list('close').over(win26)) \
  .withColumn('win12_close_list', collect_list('close').over(win12))

base_data_tmp_ema_2 = base_data_tmp_ema_1.select(
  "*",
  ema_udf(base_data_tmp_ema_1["win26_close_list"]).alias("EMA26"),
  ema_udf(base_data_tmp_ema_1["win12_close_list"]).alias("EMA12")
)

base_data_tmp_ema_2.createOrReplaceTempView("base_data_tmp_ema_2")
#base_data_tmp_ema_2.show(Console_Preview_Limit)

In [0]:
#
# PPO BERECHNUNG
#

base_data_tmp_ema_3 = spark.sql(
  " select *, (((EMA12 - EMA26) / EMA26) * CAST(100 AS DOUBLE))   " \
  " FROM base_data_tmp_ema_2 " \
  " order by filename asc, date desc "
)

#
# SIGNAL BERECHNUNG
#
base_data_tmp_ema_4 = base_data_tmp_ema_3 \
  .withColumn('win9_ema12sub26_list', collect_list('(((EMA12 - EMA26) / EMA26) * CAST(100 AS DOUBLE))').over(win9))


base_data_tmp_ema_5 = base_data_tmp_ema_4.select(
  "*",
  base_data_tmp_ema_4['(((EMA12 - EMA26) / EMA26) * CAST(100 AS DOUBLE))'].alias("PPO"),
  ema_udf(base_data_tmp_ema_4["win9_ema12sub26_list"]).alias("SIGNAL")
)

base_data_tmp_ema_5.createOrReplaceTempView("base_data_tmp_ema_5")
#base_data_tmp_ema_5.show(Console_Preview_Limit)

*PPO_Histogram-Berechnung*
---
---

In [0]:
#
# PPO-HISTOGRAM BERECHNUNG
#

base_data_tmp_ema_6 = spark.sql(
  " select *, (PPO - SIGNAL)   " \
  " FROM base_data_tmp_ema_5 " \
  " order by filename asc, date desc "
)

base_data_ema = base_data_tmp_ema_6.select(
  "*",
  base_data_tmp_ema_6['(PPO - SIGNAL)'].alias("PPO_HISTOGRAM")
)

base_data_ema.createOrReplaceTempView("base_data_ema")
#base_data_ema.show(Console_Preview_Limit)

---
# Analyse 
---
*PPO-Analyse ebenfalls basierend auf: [Investopedia.com](https://www.investopedia.com/terms/p/ppo.asp#what-the-indicator-tells-you)*


*Windows*
---
---

In [0]:
#
# Analyse-Windows
#

win2 = (Window
    .partitionBy("filename")
    .orderBy("date") \
    .rowsBetween(-1, 0))

win3 = (Window
    .partitionBy("filename")
    .orderBy("date")
    .rowsBetween(-2, 0))

*Funktionen*
---
---
**→ ppo_trend_analysis** :

Auf den 'PPO' angewendet (window mind. 1):

> *When the PPO is above zero, that helps confirm an uptrend since the short-term EMA is above the longer-term EMA. When the PPO is below zero, the short-term EMA is below the longer-term EMA, which is an indication of a downtrend.*


**→ ppo_crossover_signal_analysis** :

Auf den 'PPO_HISTOGRAM' angewendet (window mind. 2):
> *The indicator generates a buy signal when the PPO line crosses above the signal line from below, and a sell signal occurs when the PPO line crosses  below the signal from above.*

Auf den 'PPO' angewendet (window mind. 2):
> *Centerline crossovers also generate trading signals. Traders consider a move from below to above the centerline as bullish, and a move from above to below the centerline as bearish.*

**→ ppo_technical_divergence_analysis:**

Auf den 'close' und 'PPO' (window mind. 2; ggf. 'close' mit 'open' tauschen):
> *Traders can also use the PPO to look for technical divergence between the indicator and price. For example, if the price of an asset makes a higher high, but the indicator makes a lower high, it may indicate the upward momentum is subsiding. Conversely, if an asset's price makes a lower low, but the indicator makes a higher low, it could suggest that the bears are losing their traction and the price could head higher soon.*


In [0]:
#
# Analyse-UDF
#

def ppo_trend_analysis(ar):
    output = ""
    count = len(ar)
    if count > 0:
      # Einzelabgleich
      if count == 1:
        current = ar[0]
        if current > 0:
          output = "▲ Aufwärtstrend" # "Aufwärtstrend"
        elif current < 0:
          output = "▼ Abwärtstrend" # "Abwärtstrend"

      # Vergleich mit den davorherigenden; ggf. mit mehreren vorherigen
      elif count > 1:
        last_position = count - 1
        next_to_last_position = last_position - 1

        last = ar[last_position] 
        next_to_last = ar[next_to_last_position]

        if last > 0 and next_to_last > 0:
          output = "▲ Aufwärtstrend"
        elif last < 0 and next_to_last > 0:
          output = "↘ neuer Abwärtstrend"
        elif last < 0 and next_to_last < 0:
          output = "▼ Abwärtstrend"
        elif last > 0 and next_to_last < 0:
          output = "↗ neuer Aufwärtstrend"

    return output

def ppo_crossover_signal_analysis(ar):
    output = ""
    count = len(ar)
    # Vergleich des letzten Crossovers
    if count > 1:
        last_position = count - 1
        next_to_last_position = last_position - 1
        
        last = ar[last_position] 
        next_to_last = ar[next_to_last_position]

        if last <= 0 and next_to_last > 0:
          output = "Verkauf-Signal"
        elif last >= 0 and next_to_last < 0:
          output = "Kauf-Signal"

    return output

def ppo_technical_divergence_analysis(close_ar, ppo_ar):
    output = ""
    count_close = len(close_ar)
    count_ppo = len(ppo_ar)
    
    # Arrays müssen selbe Größe haben
    if count_close == count_ppo and count_close > 1:
      # delta berechnen
      pointer = count_close - 1
      delta_close = close_ar[pointer]
      delta_ppo = ppo_ar[pointer]
      pointer -= 1

      while pointer >= 0:
        delta_close -= close_ar[pointer]
        delta_ppo -= ppo_ar[pointer]
        pointer -= 1

      # steigung berechnen
      div_close = delta_close / count_close
      div_ppo = delta_ppo / count_ppo

      if div_close > div_ppo:
        output = "↧ Abflachend"
      else:
        output = "↥ Wachsend"

    return output

ppo_trend_analysis_udf = udf(ppo_trend_analysis, StringType())
ppo_crossover_signal_analysis_udf = udf(ppo_crossover_signal_analysis, StringType())
ppo_technical_divergence_analysis_udf = udf(ppo_technical_divergence_analysis, StringType())

*Aggregation*
---
---

In [0]:
#
# SIGNAL ANALYSIS
#

base_data_ema_tmp_eval = base_data_ema \
  .withColumn('win2_ppo_list', collect_list('PPO').over(win2)) \
  .withColumn('win2_ppoh_list', collect_list('PPO_Histogram').over(win2)) \
  .withColumn('win2_close_list', collect_list('close').over(win2))

base_data_ema_eval_result = base_data_ema_tmp_eval.select(
  "*",
  ppo_trend_analysis_udf(base_data_ema_tmp_eval["win2_ppo_list"]).alias("TREND"),
  ppo_crossover_signal_analysis_udf(base_data_ema_tmp_eval["win2_ppo_list"]).alias("WEAK_SIGNAL"),
  ppo_crossover_signal_analysis_udf(base_data_ema_tmp_eval["win2_ppoh_list"]).alias("STRONG_SIGNAL"),
  ppo_technical_divergence_analysis_udf(base_data_ema_tmp_eval["win2_close_list"], base_data_ema_tmp_eval["win2_ppo_list"]).alias("DIVERGENCE")
)

base_data_ema_eval_result.createOrReplaceTempView("base_data_ema_eval_result")
#base_data_ema_eval_result.show(Console_Preview_Limit)

---
# Auswertung
---

*Für alle finalen Empfehlungen gilt eine Reichteweite von 0 (schwach) bis 3 (stark)*

**Halten**
---
---
*   **H0:** 
  * ▼ Abwärtstrend && ↧ Abflachend
  * ▼ Abwärtstrend && ↥ Wachsend
  * ↘ neuer Abwärtstrend && ↧ Abflachend
*   **H1:**
  * ↗ neuer Aufwärtstrend && ↧ Abflachend
*   **H2:**
  * ▲ Aufwärtstrend && ↧ Abflachend
  * ↘ neuer Abwärtstrend && ↥ Wachsend
*   **H3:**
  * ▲ Aufwärtstrend && ↥ Wachsend
  * ↗ neuer Aufwärtstrend && ↥ Wachsend

In [0]:
#
# Halten-SQL-Statement
#

holdingCaseStatement = "CASE WHEN (TREND = \"↗ neuer Aufwärtstrend\" AND DIVERGENCE = \"↧ Abflachend\") THEN \"H1\"" +"\n" \
"WHEN (TREND = \"▲ Aufwärtstrend\" = \"↧ Abflachend\") OR (TREND = \"↘ neuer Abwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\") THEN \"H2\"" +"\n" \
"WHEN (TREND = \"▲ Aufwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\") OR (TREND = \"↗ neuer Aufwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\") THEN \"H3\"" +"\n" \
"ELSE \"H0\" END" +"\n"

**Kaufen**
---
---
*   **K0:** 
  * ▼ Abwärtstrend && ↧ Abflachend
  * ↘ neuer Abwärtstrend && ↧ Abflachend
*   **K1:**
  * ▼ Abwärtstrend && ↥ Wachsend
  * ↘ neuer Abwärtstrend && ↥ Wachsend
  * ↗ neuer Aufwärtstrend && ↧ Abflachend
*   **K2:**
  * ▲ Aufwärtstrend && ↧ Abflachend
  * ↗ neuer Aufwärtstrend && ↥ Wachsend
*   **K3:**
  * ▲ Aufwärtstrend && ↥ Wachsend

In [0]:
#
# Kaufen-SQL-Statement
#

buyingCaseStatement = "CASE WHEN (TREND = \"▼ Abwärtstrend\" AND DIVERGENCE = \"↧ Abflachend\") OR (TREND = \"↘ neuer Abwärtstrend\" AND DIVERGENCE = \"↧ Abflachend\") THEN \"K0\"" +"\n" \
"WHEN (TREND = \"▲ Aufwärtstrend\" AND DIVERGENCE = \"↧ Abflachend\") OR (TREND = \"↗ neuer Aufwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\") THEN \"K2\"" +"\n" \
"WHEN (TREND = \"▲ Aufwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\") THEN \"K3\"" +"\n" \
"ELSE \"K1\" END" +"\n"

**Verkaufen**
---
---
*   **V0:** 
  * ▲ Aufwärtstrend && ↥ Wachsend
  * ▲ Aufwärtstrend && ↧ Abflachend
*   **V1:**
  * ↗ neuer Aufwärtstrend && ↥ Wachsend
  * ↘ neuer Abwärtstrend && ↥ Wachsend
*   **V2:**
  * ▼ Abwärtstrend && ↥ Wachsend
  * ↗ neuer Aufwärtstrend && ↧ Abflachend
  * ↘ neuer Abwärtstrend && ↧ Abflachend
*   **V3:**
  * ▼ Abwärtstrend && ↧ Abflachend

In [0]:
#
# Verkaufen-SQL-Statement
#

sellingCaseStatement = "CASE WHEN (TREND = \"▲ Aufwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\")OR (TREND = \"▲ Aufwärtstrend\" AND DIVERGENCE = \"↧ Abflachend\") THEN \"V0\"" +"\n" \
"WHEN (TREND = \"↗ neuer Aufwärtstrend\" = \"↥ Wachsend\" AND DIVERGENCE = \"↥ Wachsend\") OR (TREND = \"↘ neuer Abwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\") THEN \"V1\"" +"\n" \
"WHEN (TREND = \"▼ Abwärtstrend\" AND DIVERGENCE = \"↧ Abflachend\") THEN \"V3\" " +"\n" \
"ELSE \"V2\" END" +"\n"

**Aggregation**
---
---
*vgl. dazu Investopedia-Link (s.h. Sections-Header)*

* Wenn beide Signale null sind ⇒ *halten*
* Ansonsten 'Strong-' > 'Weaksignal', für jeweils beide gilt:
  * KAUF-Signal ⇒ *kaufen*
  * VERKAUF-Signal ⇒ *verkaufen*

```
+----------------+----------------+------------+
|  WEAK_SIGNAL   | STRONG_SIGNAL  | EMPFEHLUNG |
+----------------+----------------+------------+
| null           | null           | HALTEN     |
| Kauf-Signal    | null           | KAUFEN     |
| Kauf-Signal    | Kauf-Signal    | KAUFEN     |
| ...            | ...            | ...        |
| Verkauf-Signal | null           | VERKAUFEN  |
| Verkauf-Signal | Verkauf-Signal | VERKAUFEN  |
| ...            | ...            | ...        |
| Kauf-Signal    | Verkauf-Signal | VERKAUFEN  |
| Verkauf-Signal | Kauf-Signal    | KAUFEN     |
+----------------+----------------+------------+
```


In [0]:
#
# Combiniertes-SQL-Statement
#

final_result_statement = "SELECT *," \
" CASE" \
"   WHEN (WEAK_SIGNAL = \"\" AND STRONG_SIGNAL = \"\") THEN " + holdingCaseStatement + \
"   WHEN (STRONG_SIGNAL = \"\") THEN " \
"     CASE" \
"        WHEN (WEAK_SIGNAL = \"Kauf-Signal\") THEN " + buyingCaseStatement + \
"        ELSE " + sellingCaseStatement + \
"     END" \
"   ELSE" \
"     CASE" \
"        WHEN (STRONG_SIGNAL = \"Kauf-Signal\") THEN " + buyingCaseStatement + \
"        ELSE " + sellingCaseStatement + \
"     END"  \
" END AS PPO_RESULT" \
" FROM base_data_ema_eval_result"

data_final_eval_result = spark.sql(final_result_statement)
data_final_eval_result.createOrReplaceTempView("data_final_eval_result")

---
# Visualisierung
---

*Parameter*
---

In [20]:
#
# Minimales bis Maximale Daten der geladenen Kruse
#

base_data_dayfiltered_informationView.show()

+----------+--------+-------------------+-------------------+
|  filename|count(1)|          min(date)|          max(date)|
+----------+--------+-------------------+-------------------+
|btcust.csv|     192|2019-03-11 23:54:00|2019-10-01 23:35:00|
|btcusd.csv|    2211|2013-04-01 23:58:00|2019-10-01 18:46:00|
+----------+--------+-------------------+-------------------+



In [0]:
#@markdown Parameter beziehen sich lediglich auf die Ausgabe:
Anzeige_Limit = 365  #@param {type: "slider", min: 1, max: 10000}
Anzeige_Zahlen_Genaugikeit = 3  #@param {type: "slider", min: 1, max: 10}
Anzeige_Template = "plotly_dark" #@param ['ggplot2', 'seaborn', 'simple_white', 'plotly', 'plotly_white', 'plotly_dark', 'presentation', 'xgridoff', 'ygridoff', 'gridon', 'none'] {allow-input: false}
#@markdown Start- & End-Tag haben keinen Einfluss auf die Exportierten Tage:
Anzeige_Start_Tag = '2018-10-01' #@param {type:"date"}
Anzeige_Ende_Tag = '2019-10-01' #@param {type:"date"}

Anzeige_Zahlen_Genaugikeit_str = str(Anzeige_Zahlen_Genaugikeit)

*Vorbereitung*
---

In [0]:
#
# Datavorbereitung
#

data_eval_result = spark.sql(
  " select filename, Date(date), format_number(high, "+Anzeige_Zahlen_Genaugikeit_str+") high, format_number(low, "+Anzeige_Zahlen_Genaugikeit_str+") low, format_number(open, "+Anzeige_Zahlen_Genaugikeit_str+") open, format_number(close, "+Anzeige_Zahlen_Genaugikeit_str+") close" \
  " , format_number(PPO, "+Anzeige_Zahlen_Genaugikeit_str+") PPO, format_number(SIGNAL, "+Anzeige_Zahlen_Genaugikeit_str+") Signal, format_number(PPO_HISTOGRAM, "+Anzeige_Zahlen_Genaugikeit_str+") PPO_HISTOGRAM " \
  " , TREND, WEAK_SIGNAL, STRONG_SIGNAL, DIVERGENCE, PPO_RESULT "\
  " FROM data_final_eval_result " \
  " WHERE date >= '"+ Anzeige_Start_Tag.strip() +" 00:00:00' AND date <= '"+ Anzeige_Ende_Tag.strip() +"23:59:59' " \
  " order by filename asc, date asc " 
)

data_pd = data_eval_result.toPandas()

# Tausender-Zeichen entfernen: 1,000.001 -> 1000.001
data_pd.close = (data_pd['close'].replace('\,','', regex = True).astype(float))
data_pd.open = (data_pd['open'].replace('\,','', regex = True).astype(float))
data_pd.high = (data_pd['high'].replace('\,','', regex = True).astype(float))
data_pd.low = (data_pd['low'].replace('\,','', regex = True).astype(float))

data_pd.date = pd.to_datetime(data_pd.date)
data_pd.close = pd.to_numeric(data_pd.close)
data_pd.PPO = pd.to_numeric(data_pd.PPO)
data_pd.Signal = pd.to_numeric(data_pd.Signal)
data_pd.PPO_HISTOGRAM = pd.to_numeric(data_pd.PPO_HISTOGRAM)

data_pd.filename = (data_pd['filename']).astype(str)

# Vorbereiten der for-each-loop
data_pd_filenames = spark.sql("SELECT DISTINCT filename FROM data_final_eval_result").toPandas().to_numpy()

Output
---

In [24]:
#
# Charts & Table
#

hideInjectedFilenameColumn_CSS = "<style>" \
  "table td:nth-child(1) { display:none;}" \
  "table.dataframe thead th:first-child {display: none;}" \
  "</style>"

for currentFilenames in data_pd_filenames:
  for currentFilename in currentFilenames:

      current_data_pd = data_pd[data_pd.filename == currentFilename]

      stock_chart = go.Figure(
          data = [ go.Candlestick( 
              x = current_data_pd['date'],
              open = current_data_pd['open'],
              high = current_data_pd['high'],
              low = current_data_pd['low'],
              close = current_data_pd['close']
              )]
      )

      ppo_chart = go.Figure(
          data = [ go.Scatter(
              x = current_data_pd['date'],
              y = current_data_pd['PPO'],
              mode = 'lines',
              name = 'PPO'
          ), go.Scatter(
              x = current_data_pd['date'],
              y = current_data_pd['Signal'],
              mode = 'lines',
              name = 'Signal'
          )]
      )

      ppo_chart.add_trace(
          go.Bar(
              name = 'PPO-Histogram',
              x = current_data_pd['date'],
              y = current_data_pd['PPO_HISTOGRAM']
          )
      )

      stock_chart.update_layout(
          title='Chart für CSV-Datei \'' + currentFilename + '\'',
          yaxis_title = 'Points',
          template = Anzeige_Template,
          xaxis_rangeslider_visible = False
      )

      ppo_chart.update_layout(
          title='PPO-Indikator für CSV-Datei \'' + currentFilename + '\'',
          yaxis_title = 'Changes in %',
          template = Anzeige_Template
      )

      print("\n\nFilename: " + currentFilename + "\nZeitraum: " + Anzeige_Start_Tag + " → " + Anzeige_Ende_Tag + "\n")
      stock_chart.show()
      ppo_chart.show()
      print()
      display(HTML(data_pd.to_html(index = False, max_rows = Anzeige_Limit) + hideInjectedFilenameColumn_CSS))



Filename: btcust.csv
Zeitraum: 2018-10-01 → 2019-10-01



filename,date,high,low,open,close,PPO,Signal,PPO_HISTOGRAM,TREND,WEAK_SIGNAL,STRONG_SIGNAL,DIVERGENCE,PPO_RESULT
btcusd.csv,2018-10-01,6601.900,6600.800,6600.800,6601.900,0.851,-0.691,1.542,▲ Aufwärtstrend,,,↥ Wachsend,H3
btcusd.csv,2018-10-02,6538.300,6527.500,6538.300,6527.500,1.513,-0.169,1.682,▲ Aufwärtstrend,,,↥ Wachsend,H3
btcusd.csv,2018-10-03,6503.200,6503.049,6503.200,6503.200,1.628,0.171,1.457,▲ Aufwärtstrend,,,↥ Wachsend,H3
btcusd.csv,2018-10-04,6590.400,6590.350,6590.350,6590.400,1.358,0.365,0.992,▲ Aufwärtstrend,,,↧ Abflachend,H0
btcusd.csv,2018-10-05,6641.500,6639.900,6641.500,6639.900,0.898,0.480,0.418,▲ Aufwärtstrend,,,↧ Abflachend,H0
btcusd.csv,2018-10-06,6595.200,6592.800,6592.800,6595.200,0.603,0.473,0.130,▲ Aufwärtstrend,,,↥ Wachsend,H3
btcusd.csv,2018-10-07,6609.400,6608.886,6608.900,6609.400,0.565,0.515,0.050,▲ Aufwärtstrend,,,↧ Abflachend,H0
btcusd.csv,2018-10-08,6673.300,6673.200,6673.300,6673.200,0.787,0.789,-0.001,▲ Aufwärtstrend,,Verkauf-Signal,↧ Abflachend,V0
btcusd.csv,2018-10-09,6663.000,6663.000,6663.000,6663.000,0.683,0.851,-0.168,▲ Aufwärtstrend,,,↥ Wachsend,H3
btcusd.csv,2018-10-10,6630.300,6630.000,6630.300,6630.074,0.499,0.869,-0.370,▲ Aufwärtstrend,,,↥ Wachsend,H3




Filename: btcusd.csv
Zeitraum: 2018-10-01 → 2019-10-01



filename,date,high,low,open,close,PPO,Signal,PPO_HISTOGRAM,TREND,WEAK_SIGNAL,STRONG_SIGNAL,DIVERGENCE,PPO_RESULT
btcusd.csv,2018-10-01,6601.900,6600.800,6600.800,6601.900,0.851,-0.691,1.542,▲ Aufwärtstrend,,,↥ Wachsend,H3
btcusd.csv,2018-10-02,6538.300,6527.500,6538.300,6527.500,1.513,-0.169,1.682,▲ Aufwärtstrend,,,↥ Wachsend,H3
btcusd.csv,2018-10-03,6503.200,6503.049,6503.200,6503.200,1.628,0.171,1.457,▲ Aufwärtstrend,,,↥ Wachsend,H3
btcusd.csv,2018-10-04,6590.400,6590.350,6590.350,6590.400,1.358,0.365,0.992,▲ Aufwärtstrend,,,↧ Abflachend,H0
btcusd.csv,2018-10-05,6641.500,6639.900,6641.500,6639.900,0.898,0.480,0.418,▲ Aufwärtstrend,,,↧ Abflachend,H0
btcusd.csv,2018-10-06,6595.200,6592.800,6592.800,6595.200,0.603,0.473,0.130,▲ Aufwärtstrend,,,↥ Wachsend,H3
btcusd.csv,2018-10-07,6609.400,6608.886,6608.900,6609.400,0.565,0.515,0.050,▲ Aufwärtstrend,,,↧ Abflachend,H0
btcusd.csv,2018-10-08,6673.300,6673.200,6673.300,6673.200,0.787,0.789,-0.001,▲ Aufwärtstrend,,Verkauf-Signal,↧ Abflachend,V0
btcusd.csv,2018-10-09,6663.000,6663.000,6663.000,6663.000,0.683,0.851,-0.168,▲ Aufwärtstrend,,,↥ Wachsend,H3
btcusd.csv,2018-10-10,6630.300,6630.000,6630.300,6630.074,0.499,0.869,-0.370,▲ Aufwärtstrend,,,↥ Wachsend,H3


----
# CSV-Export
----
*Die Daten sollen nach Tage sotiert ausgegeben werden (vgl. Besprechung 13.12.19)*

*Darstellung des Datums nach [ISO-8601-Norm](https://lmgtfy.com/?q=ISO-8601) (z.B.: 2019-09-07)*

Ordner-Struktur
---

**aktueller Pfad sollte ~/content' sein**

In [0]:
# maximal einen Unterordner
Export_Pfad = 'ABDA2019/spark_warehouse/'  #@param {type: "string"}
Export_Ordnername = 'ppo_csv_export'  #@param {type: "string"}

In [0]:
#
# (Neuen) Ordner Erstellen
#

# Alten Ordner löschen
if os.path.exists(os.path.join(Export_Pfad, Export_Ordnername)):
  os.rmdir(os.path.join(Export_Pfad, Export_Ordnername))

# Zwischen Ordner-Checken
if not os.path.exists(Export_Pfad):
  os.mkdir(Export_Pfad)

# Export Ordner erstellen
os.mkdir(os.path.join(Export_Pfad, Export_Ordnername))

Vorbereitung
---

In [0]:
#
# Outputt Daten sammeln
#

data_output = spark.sql(
  " select filename, date, open, close, high, low, volume, PPO_RESULT" \
  " FROM data_final_eval_result " \
  " order by filename asc, date asc " 
).toPandas()

# Types definieren
data_output.filename = (data_output['filename']).astype(str)
data_output.date = pd.to_datetime(data_output.date)
data_output.close = (data_output['close'].replace('\,','', regex = True).astype(float))
data_output.open = (data_output['open'].replace('\,','', regex = True).astype(float))
data_output.high = pd.to_numeric(data_output.high)
data_output.low = pd.to_numeric(data_output.low)
data_output.volume = pd.to_numeric(data_output.volume)
data_output.PPO_RESULT = (data_output['PPO_RESULT']).astype(str)

data_output_days = spark.sql("SELECT DISTINCT date FROM data_final_eval_result").toPandas()
data_output_days.date = data_output_days.date = pd.to_datetime(data_output_days.date)

data_distinct_days = data_output_days.to_numpy()

Export
---

In [0]:
#
# Daten als csv exportieren
#

from pandas import Timestamp

for currentDays in data_distinct_days:
  for currentDay in currentDays:

      currentOutput = data_output[data_output.date == currentDay]
      currentTimestamp = Timestamp(currentDay)

      if currentTimestamp.month > 9:
        formattedMonth = str(currentTimestamp.month)
      else:
        formattedMonth = '0' + str(currentTimestamp.month)

      if currentTimestamp.day > 9:
        formattedDay = str(currentTimestamp.day)
      else:
        formattedDay = '0' + str(currentTimestamp.day)

      currentFilename = str(currentTimestamp.year) + "-" + formattedMonth + "-" + formattedDay + ".csv"

      if not os.path.exists(os.path.join(Export_Pfad, Export_Ordnername)):
        currentOutput.to_csv(currentFilename, sep='\t', encoding='utf-8', index=False)
      else:
        currentOutput.to_csv(os.path.join(Export_Pfad, Export_Ordnername, currentFilename), sep='\t', encoding='utf-8', index=False)

Output
---

In [29]:
#
# Output
#

print("Export abgeschlossen.")
if os.path.exists(os.path.join(Export_Pfad, Export_Ordnername)):
  os.chdir(os.path.join(Export_Pfad, Export_Ordnername))
!ls

Export abgeschlossen.
2013-04-01.csv	2014-06-18.csv	2015-11-16.csv	2017-04-09.csv	2018-07-20.csv
2013-04-02.csv	2014-06-19.csv	2015-11-17.csv	2017-04-10.csv	2018-07-21.csv
2013-04-03.csv	2014-06-20.csv	2015-11-18.csv	2017-04-11.csv	2018-07-22.csv
2013-04-04.csv	2014-06-21.csv	2015-11-19.csv	2017-04-12.csv	2018-07-23.csv
2013-04-05.csv	2014-06-22.csv	2015-11-20.csv	2017-04-13.csv	2018-07-24.csv
2013-04-06.csv	2014-06-23.csv	2015-11-21.csv	2017-04-14.csv	2018-07-25.csv
2013-04-07.csv	2014-06-24.csv	2015-11-22.csv	2017-04-15.csv	2018-07-26.csv
2013-04-08.csv	2014-06-25.csv	2015-11-23.csv	2017-04-16.csv	2018-07-27.csv
2013-04-09.csv	2014-06-26.csv	2015-11-24.csv	2017-04-17.csv	2018-07-28.csv
2013-04-10.csv	2014-06-27.csv	2015-11-25.csv	2017-04-18.csv	2018-07-29.csv
2013-04-11.csv	2014-06-28.csv	2015-11-26.csv	2017-04-19.csv	2018-07-30.csv
2013-04-12.csv	2014-06-29.csv	2015-11-27.csv	2017-04-20.csv	2018-07-31.csv
2013-04-13.csv	2014-06-30.csv	2015-11-28.csv	2017-04-21.csv	2018-08-01.csv
201

---
# Vergleichbare Charts
---

**BTCUSD**: [traidingview.com](https://de.tradingview.com/chart/ecCxGiMv/) *(login erforderlich)*

```
+-------------+------------+------------+-------------+-------------+
| TraidingView.com:                                                 |
+-------------+------------+------------+-------------+-------------+
|             | 2019-10-01 | 2019-09-30 | 2019-08-10  | 2019-08-07  |
+-------------+------------+------------+-------------+-------------+
| EMA26       |    9305.74 |    9384.68 |    10848.14 |    10630.29 |
| EMA12       |    8734.78 |    8810.38 |    11193.74 |    10886.97 |
| PPO         |      -6.14 |      -6.12 |        3.19 |        2.41 |
| SIGNAL      |      -4.56 |      -4.17 |        1.63 |        0.13 |
| PPO_History |      -1.57 |      -1.95 |        1.55 |        2.28 |
+-------------+------------+------------+-------------+-------------+

+-------------+------------+------------+-------------+-------------+
| CALCULATED:                                                       |
+-------------+------------+------------+-------------+-------------+
|             | 2019-10-01 | 2019-09-30 | 2019-08-10  | 2019-08-07  |
+-------------+------------+------------+-------------+-------------+
| EMA26       |  9327.3017 |  9446.6421 |  10651.5067 |  10673.4093 |
| EMA12       |  8751.5960 |  8842.4748 |  11163.8865 |  10785.1409 |
| PPO         |     -6.172 |    -6.3956 |      4.8104 |      1.0468 |
| SIGNAL      |    -4.4147 |    -4.0178 |      0.9102 |     -1.8162 |
| PPO_History |    -1.7576 |    -2.3778 |      3.9002 |      2.8631 |
+-------------+------------+------------+-------------+-------------+
```



**intc.csv (utf-8)** *--> eher ungeeignetes Beispiel*

 see [School.StockCharts.com](https://school.stockcharts.com/doku.php?id=technical_indicators:price_oscillators_ppo) 

```
time,open,close,high,low,volume
1271203140000,21.16,20.16,20.16,20.16,0.0
1271289540000,20.49,20.49,20.49,20.49,0.0
1271375940000,20.74,20.74,20.74,20.74,0.0
1271462340000,20.77,20.77,20.77,20.77,0.0
1271721540000,20.53,20.53,20.53,20.53,0.0
1271807940000,19.61,19.61,19.61,19.61,0.0
1271894340000,20.02,20.02,20.02,20.02,0.0
1271980740000,19.70,19.70,19.70,19.70,0.0
1272067140000,19.94,19.94,19.94,19.94,0.0
1272326340000,19.62,19.62,19.62,19.62,0.0
1272412740000,19.11,19.11,19.11,19.11,0.0
1272499140000,19.32,19.32,19.32,19.32,0.0
1272585540000,19.61,19.61,19.61,19.61,0.0
1272671940000,19.54,19.54,19.54,19.54,0.0
1272931140000,18.89,18.89,18.89,18.89,0.0
1273017540000,19.33,19.33,19.33,19.33,0.0
1273103940000,19.21,19.21,19.21,19.21,0.0
1273190340000,19.51,19.51,19.51,19.51,0.0
1273276740000,19.55,19.55,19.55,19.55,0.0
1273535940000,19.92,19.92,19.92,19.92,0.0
1273622340000,20.29,20.29,20.29,20.29,0.0
1273708740000,20.58,20.58,20.58,20.58,0.0
1273795140000,20.52,20.52,20.52,20.52,0.0
1273881540000,20.69,20.69,20.69,20.69,0.0
1274140740000,20.67,20.67,20.67,20.67,0.0
1274227140000,20.72,20.72,20.72,20.72,0.0
1274313540000,20.25,20.25,20.25,20.25,0.0
1274399940000,20.56,20.56,20.56,20.56,0.0
1274486340000,20.49,20.49,20.49,20.49,0.0
1274745540000,20.39,20.39,20.39,20.39,0.0

```

